In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
import moabb

m_dataset = moabb.datasets.bi2013a(
    NonAdaptive=True,
    Adaptive=True,
    Training=True,
    Online=True,
)

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
2024-03-24 17:39:10.551859: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 17:39:10.551929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempt

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


In [3]:
m_dataset.download()

MNE_DATA is not already configured. It will be set to default location in the home directory - /home/void/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location
Attempting to create new mne-python configuration file:
/home/void/.mne/mne-python.json


/home/void/.local/lib/python3.11/site-packages/moabb/datasets/download.py:55: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BRAININVADERS2013_PATH"
  set_config(key, get_config("MNE_DATA"))
/usr/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zenodo.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zenodo.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|████████████████████████████████████████| 265M/265M [00:00<00:00, 522GB/s]
SHA256 hash of downloaded file: 8c0c9cb3e3345247430d5945e91c645bffa4074f79a196ac911aefde67a0acc1
Use this value 

In [5]:
m_data = m_dataset.get_data()

In [7]:
m_data.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [10]:
import sys
sys.getsizeof(m_data)

1168

In [12]:
import numpy as np
from scipy import signal
from sklearn.base import BaseEstimator, TransformerMixin


class Transformer(BaseEstimator, TransformerMixin):
    '''
    Base class for transformers providing dummy implementation
        of the methods expected by sklearn
    '''
    def fit(self, x, y=None):
        return self


class ButterFilter(Transformer):
    '''Applies Scipy's Butterworth filter'''
    def __init__(self, sampling_rate: int, order: int, highpass: int, lowpass: int) -> None:
        self.sampling_rate = sampling_rate
        self.order = order
        self.highpass = highpass
        self.lowpass = lowpass

        normal_cutoff = [a / (0.5 * self.sampling_rate) for a in (self.highpass, self.lowpass)]
        self.filter = signal.butter(self.order, normal_cutoff, btype='bandpass')

    def transform(self, x):
        out = np.empty_like(x)
        out[:] = [signal.filtfilt(*self.filter, item) for item in x]
        return out


class Decimator(Transformer):
    def __init__(self, factor: int):
        '''
        factor: downsampling factor, shouldn't be more than 13,
            see :py:funct:`scipy.signal.decimate` for more info
        '''
        self.factor = factor

    def transform(self, x):
        '''
        Args:
            x: iterable of np.ndarrays
        Returns:
            np.ndarray of np.objects shaped (len(x), )
                In other words it outputs ndarray of objects each of which is
                result of decimation of items from x
        '''
        out = np.empty(len(x), dtype=np.object)
        out[:] = [signal.decimate(item, self.factor) for item in x]
        return out


class ChannellwiseScaler(Transformer):
    '''Performs channelwise scaling according to given scaler
    '''
    def __init__(self, scaler: Transformer):
        '''Args:
            scaler: instance of one of sklearn.preprocessing classes
                StandardScaler or MinMaxScaler or analogue
        '''
        self.scaler = scaler

    def fit(self, x: np.ndarray, y=None):
        '''
        Args:
            x: array of eegs, that is every element of x is (n_channels, n_ticks)
                x shaped (n_eegs) of 2d array or (n_eegs, n_channels, n_ticks)
        '''
        for signals in x:
            self.scaler.partial_fit(signals.T)
        return self

    def transform(self, x):
        '''Scales each channel

        Wors either with one record, 2-dim input, (n_channels, n_samples)
            or many records 3-dim, (n_records, n_channels, n_samples)
        Returns the same format as input
        '''
        scaled = np.empty_like(x)
        for i, signals in enumerate(x):
            # double T for scaling each channel separately
            scaled[i] = self.scaler.transform(signals.T).T
        return scaled


class MarkersTransformer(Transformer):
    '''Transforms markers channels to arrays of indexes of epoch start and labels
    '''
    def __init__(self, labels_mapping: dict, decimation_factor: int=1, empty_label: float=0.):
        self.labels_mapping = labels_mapping
        self.decimation_factor = decimation_factor
        self.empty_label = empty_label

    def transform(self, batch):
        res = []
        for markers in batch:
            index_label = []
            for index, label in enumerate(markers):
                if label == self.empty_label: continue
                index_label.append([
                    index // self.decimation_factor,
                    self.labels_mapping[label],
                ])
            res.append(np.array(index_label, dtype=np.int))
        return res


In [13]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import mne


def mne_arrays(
    bin_epochs: list,
    rate: int,
    ch_set: list,
):
    info = mne.create_info(
        ch_names=ch_set,
        ch_types=['eeg']*len(ch_set),
        sfreq=rate,
        montage=mne.channels.read_montage(kind='easycap-M1'),
    )

    epochs = np.concatenate([i[0] for i in bin_epochs])
    events = np.concatenate([i[1] for i in bin_epochs])
    events = np.c_[np.arange(len(events)), events, events]

    epochs_array = mne.EpochsArray(data=epochs, info=info, events=events, verbose=False)
    evoked_nontarget = epochs_array['0'].average()
    evoked_target_raw = copy.deepcopy(epochs_array['1']).average()

    epochs_delta = epochs_array['1'].subtract_evoked(evoked=evoked_nontarget)
    evoked_target_delta = copy.deepcopy(epochs_delta).average()

    return {
        'target': evoked_target_raw,
        'nontarget': evoked_nontarget,
        'delta': evoked_target_delta,
    }


def plot_evoked_electrodes(
    bin_epochs: list,
    rate: int,
    ch_set: list,
    title: str='',
    figsize: tuple=(16, 10),
):
    '''Plots averaged evoked potentials from labeled binary epochs data

    Args:
        bin_epochs: data in format of `P300Dataset.binary_epochs`
        rate: rate of given signal
        ch_set: list of names of channels

    Returns:
        figure epochs plotted on
    '''
    colors = {
        'target': 'red',
        'nontarget': 'blue',
        'delta': 'black',
    }

    data = mne_arrays(bin_epochs, rate, ch_set)
    info = mne.create_info(
        ch_names=ch_set,
        ch_types=['eeg']*len(ch_set),
        sfreq=rate,
        montage=mne.channels.read_montage(kind='easycap-M1'),
    )

    # plot data
    fig = plt.figure(figsize=figsize)
    topo_iterator = mne.viz.topo.iter_topography(info, fig=fig, layout_scale=0.945,
                        fig_facecolor='w', axis_facecolor='w', axis_spinecolor='w')
    for axis, channel in topo_iterator:
        for name, value in data.items():
            axis.plot(value.times, value.data[channel], color=colors[name], label=name)
        axis.axhline(0, color='black', linewidth=1)
        axis.grid()
        axis.set_title(ch_set[channel])

    fig.axes[0].legend(prop={'size': 10}, loc=(-0.4, 0.1))
    fig.suptitle(title, fontsize=20)


def plot_evoked_map(
    bin_epochs: list,
    rate: int,
    ch_set: list,
):
    target = mne_arrays(bin_epochs, rate, ch_set)['target']
    target.plot_topomap()

def plot_evoked_joint(
    bin_epochs: list,
    rate: int,
    ch_set: list,
):
    target = mne_arrays(bin_epochs, rate, ch_set)['target']
    target.plot_joint()


In [25]:
print(m_data[1]['1']['0'])
# channels = m_data[1]['session_1']['run_1'].ch_names[:-1]
# channels

<RawArray | 17 x 132896 (259.6 s), ~17.3 MB, data loaded>


In [28]:
raw_dataset = []

for _, sessions in sorted(m_data.items()):
    eegs, markers = [], []
    for item, run in sorted(sessions.items()):
        for item1, run1 in sorted(run.items()):
            data = run1.get_data()
            eegs.append(data[:-1])
            markers.append(data[-1])
    raw_dataset.append((eegs, markers))